In [46]:
from pymongo import MongoClient

client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')
filter_info = client['PUBLIC']['FilterInfo'] #필터접근

f_id = 1 #input
keyid = 517 #keyid

nyear = []
pyear = []

fid_key_query = filter_info.find_one({'fId' : f_id }) #필터검색

if  f_id > 0 : #필터쿼리가 있으면
    for key in fid_key_query.keys() :
        if key ==  'nFilter':
            nyear = fid_key_query[key]['year']
            #print(nyear)
        elif key == 'pFilter' :
            pyear = fid_key_query[key]['year']
            #print(pyear)
print(nyear)
print(pyear)


[2021, 2020, 2019, 2018]
[2021, 2020, 2019, 2017]


In [47]:

scion_aut = client['SCIENCEON']['Author']
ntis_aut = client['NTIS']['Author']
dbpia_aut = client['DBPIA']['Author']

scion_autpaper = client['SCIENCEON']['AuthorPapers']
ntis_autpaper = client['NTIS']['AuthorPapers']
dbpia_autpaper = client['DBPIA']['AuthorPapers']

scion_raw = client['SCIENCEON']['Rawdata']
ntis_raw = client['NTIS']['Rawdata']
dbpia_raw = client['DBPIA']['Rawdata']

scion_key_query = scion_autpaper.find({ 'keyId' : keyid })
ntis_key_query = ntis_autpaper.find({ 'keyId' : keyid })
dbpia_key_query = dbpia_autpaper.find({ 'keyId' : keyid })

auts = [scion_aut, ntis_aut, dbpia_aut] #저자 이름, 소속
key_querys = [scion_key_query, ntis_key_query, dbpia_key_query] #a_id

a_id = []
all_name = []
all_inst = []
all_site = []
Answer_dict = {} #통합결과
Answer_pdict = {}

site = ['Scienceon', 'NTIS', 'DBPIA']

for i in range(len(key_querys)):
    for key_query in key_querys[i]: #keyid에 저자수만큼 반복
        Aid = []
        A_id = []
        all_paper = []

        a_id.append(key_query['A_ID']) #a_id read

        if site[i] == 'NTIS' : 
            ntis_raw_query = ntis_raw.find({'$and':[{'keyId':keyid},{'mngId':a_id[-1]}]})
            
            if ntis_raw_query == None:
                continue
            else:
                for raw_one in ntis_raw_query:
                    if f_id < 1 :
                        Aid = a_id[-1]
                        all_paper = key_query['papers']

                    elif int(raw_one['prdEnd'][:4]) in nyear or int(raw_one['prdStart'][:4]) in nyear : #필터링
                        Aid = a_id[-1]
                        all_paper = key_query['papers']

        elif site[i] == 'Scienceon' :
            scion_raw_query = scion_raw.find({'$and':[{'keyId':keyid},{'author_id':{'$regex':a_id[-1]}}]})
            if scion_raw_query == None :
                continue
            else:
                for raw_one in scion_raw_query:
                    if f_id < 1 :
                        A_id.append(a_id[-1])
                        for v in A_id:
                            if v not in Aid:
                                Aid.append(v)
                                all_paper = key_query['papers']

                    elif int(raw_one['issue_year'][:4]) in pyear: #필터링
                        A_id.append(a_id[-1])
                        for v in A_id:
                            if v not in Aid:
                                Aid.append(v)
                                all_paper = key_query['papers']

        elif site[i] == 'DBPIA' :
            dbpia_raw_query = dbpia_raw.find({'$and':[{'keyId':keyid},{'author_id':{'$regex':a_id[-1]}}]})
            if dbpia_raw_query == None :
                continue
            else:
                for raw_one in dbpia_raw_query:
                    if f_id < 1 :
                        A_id.append(key_query['A_ID'])
                        for v in A_id:
                            if v not in Aid:
                                Aid.append(v)
                                all_paper = key_query['papers']

                    elif int(raw_one['issue_year'][:4]) in pyear: #필터링
                        A_id.append(key_query['A_ID'])
                        for v in A_id:
                            if v not in Aid:
                                Aid.append(v)
                                all_paper = key_query['papers']
                                
        if Aid == [] or Aid == "":
            continue

        aut_query = auts[i].find_one({'_id':key_query['A_ID']}) #저자이름, 소속 read

        all_name.append(aut_query['name'])
        all_inst.append(aut_query['inst'].replace("(주) ", "").replace("(주)", "").split(' ')[0])
        all_site.append(site[i])
        
        Answer = {'name' : all_name[-1], site[i] : {'inst' : all_inst[-1], 'A_id': Aid} }
        Answer_paper = {'name' : all_name[-1], site[i] : all_paper}

        if all_name[-1] not in Answer_dict and all_name[-1]+'0' not in Answer_dict :
            Answer_dict[all_name[-1]] = Answer
            Answer_pdict[all_name[-1]] = Answer_paper
        else :
            
            count = 0
            flag = True
            while flag :
                temp = None 
                tempName = all_name[-1]
              
                if tempName in Answer_dict :        # 이름 으로만 key가ㅣ 존재         
                    temp = Answer_dict[tempName]
                    flag = False
                else :
                    tempName = all_name[-1]+str(count)  # 이름 + 숫자로 key가ㅣ 존재
                    if tempName not in Answer_dict :
                        flag = False 
                        break
                    temp = Answer_dict[tempName]
                      
                for key in temp.keys() : # 사이트 돌면서
                    if key != 'name' : 
                        src = ""
                        tgt = ""

                        if len(all_inst[-1]) >= len(temp[key]['inst']):
                            src = temp[key]['inst']
                            tgt = all_inst[-1]

                        elif len(all_inst[-1]) < len(temp[key]['inst']):
                            src = all_inst[-1]
                            tgt = temp[key]['inst']
                        #print(temp)
                        if key == site[i] :# 사이트가 동일할때
                            if temp[key]['inst'] == all_inst[-1] or (src != "" and src in tgt) :  # 소속 같을때
                                flag = False
                                break

                            elif all_name[-1]+str(count+1) not in Answer_dict : #소속이 다를 때
                                Answer_dict[all_name[-1]+str(count+1)] = Answer
                                Answer_pdict[all_name[-1]+str(count+1)] = Answer_paper

                                if tempName == all_name[-1]:
                                    Answer_dict[all_name[-1]+'0'] = temp
                                    Answer_pdict[all_name[-1]+'0'] = Answer_pdict[all_name[-1]]
                                    del Answer_dict[all_name[-1]]
                                    del Answer_pdict[all_name[-1]]

                        else :# 사이트가 다를때 
                            if temp[key]['inst'] == all_inst[-1] or (src != "" and src in tgt):  # 소속 같을때
                                    Answer_dict[tempName][site[i]] =  {'inst' : all_inst[-1], 'A_id': Aid}
                                    Answer_pdict[tempName][site[i]] = all_paper
                                    flag = False
                                    break
                            
                            elif all_name[-1]+str(count+1) not in Answer_dict :
                                Answer_dict[all_name[-1]+str(count+1)] = Answer
                                Answer_pdict[all_name[-1]+str(count+1)] = Answer_paper

                                if tempName == all_name[-1]:
                                    Answer_dict[all_name[-1]+'0'] = temp
                                    Answer_pdict[all_name[-1]+'0'] = Answer_pdict[all_name[-1]]
                                    del Answer_dict[all_name[-1]]
                                    del Answer_pdict[all_name[-1]]  
                count += 1

#print(len(a_id), len(all_name), len(all_inst), len(Answer_dict.items()))
#print(Answer_dict)
#print(sorted(Answer_dict.items()))
#print(Answer_pdict)
id_domestic = client['ID']['Domestic']

for Answer_one in Answer_dict.keys() :
    input_id = "{'fid': " + str(f_id) + ", 'keyId': " + str(keyid)
    
    if len(Answer_dict[Answer_one].keys()) == 2: #site 2개 이상이 아니면 input안함
        continue
    
    for key in Answer_dict[Answer_one].keys() :
        if key == 'Scienceon' :
            input_id +=  ", '" + key + "': " + str(Answer_dict[Answer_one][key]['A_id'])
        elif key == "NTIS" :
            input_id +=  ", '" + key + "': '" + Answer_dict[Answer_one][key]['A_id'] + "'"
        elif key == "DBPIA" :
            input_id +=  ", '" + key + "': " + str(Answer_dict[Answer_one][key]['A_id'])
    input_id += "}"

    #print(input_id)
    print(eval(input_id))
    #id_domestic.insert_one(eval(input_id))



{'fid': 1, 'keyId': 517, 'Scienceon': ['s1067155'], 'DBPIA': ['2037888']}
{'fid': 1, 'keyId': 517, 'Scienceon': ['s1067157'], 'DBPIA': ['624192']}
{'fid': 1, 'keyId': 517, 'Scienceon': ['s1067435'], 'NTIS': '11665014'}
{'fid': 1, 'keyId': 517, 'Scienceon': ['s1087957'], 'DBPIA': ['1066525']}
{'fid': 1, 'keyId': 517, 'Scienceon': ['s1087966'], 'DBPIA': ['435190']}
{'fid': 1, 'keyId': 517, 'Scienceon': ['s214929'], 'DBPIA': ['1668956']}
{'fid': 1, 'keyId': 517, 'Scienceon': ['s267323'], 'DBPIA': ['4387850']}
{'fid': 1, 'keyId': 517, 'Scienceon': ['s267665'], 'DBPIA': ['1934807']}
{'fid': 1, 'keyId': 517, 'Scienceon': ['s267666'], 'DBPIA': ['2073305']}
{'fid': 1, 'keyId': 517, 'Scienceon': ['s267667'], 'DBPIA': ['2073304']}
{'fid': 1, 'keyId': 517, 'Scienceon': ['s268621'], 'DBPIA': ['1596973']}
{'fid': 1, 'keyId': 517, 'Scienceon': ['s270157'], 'DBPIA': ['1128477']}
{'fid': 1, 'keyId': 517, 'Scienceon': ['s275733'], 'NTIS': '413606265199', 'DBPIA': ['113367']}
{'fid': 1, 'keyId': 517, 'S

In [53]:
preprocessing_p = {}
a = []
for Answerp_one in Answer_pdict.keys() :
    #print(Answer_pdict[Answerp_one].keys())
    if len(Answer_pdict[Answerp_one].keys()) == 2: #site 2개 이상이 아니면 input안함
        continue
    else:
        preprocessing_p[Answerp_one] = Answer_pdict[Answerp_one]
        a.append(Answerp_one)

a.sort()
print(a)
#print(preprocessing_p)

['강경수', '강동우', '강승록', '강은수', '강지훈0', '강지훈1', '강진구', '강철구0', '강태곤', '강태삼', '강태우', '강평', '고상호', '고석철', '고영진0', '고종선', '고희상', '공경철0', '공기석', '곽상원', '곽태희', '구본수', '구자춘0', '구자함', '국창호', '권대규', '권동수', '권민관', '권선욱', '권아람', '권혁호', '금대현', '금동석', '김가빈', '김갑기', '김건중0', '김경무0', '김경수0', '김광태0', '김국용', '김권희', '김기두', '김기선0', '김남욱', '김남호0', '김내현', '김덕관', '김도현0', '김동진0', '김동한0', '김동협', '김동환0', '김동희0', '김민창0', '김병규0', '김병기0', '김병일', '김병택', '김봉환', '김상연0', '김상우1', '김상훈0', '김상훈1', '김석원0', '김석호0', '김선희0', '김성동0', '김성민1', '김성일0', '김성한0', '김솔0', '김수정1', '김수현0', '김승권0', '김승균0', '김신후', '김연철', '김영근0', '김영길', '김영식0', '김영우0', '김영진2', '김영훈0', '김용석0', '김용재0', '김용주0', '김원호0', '김원희', '김응곤', '김이삭', '김재실', '김재혁0', '김정섭0', '김정엽0', '김정자0', '김정호0', '김종수0', '김주용0', '김중경', '김지현0', '김지훈0', '김진곤0', '김천곤', '김태욱0', '김태원0', '김태호0', '김태호2', '김학원0', '김한준0', '김현근', '김현수2', '김현주0', '김현준0', '김현진0', '김호민', '김홍집', '김희식', '김희주0', '나선익', '나정효', '남강현', '남주석', '노은솔', '노정욱', '노태성0', '류근', '류주현', '류준형', '류희경', '명현0', '목성훈', '목학수', '문민식', '문인혁

In [21]:
a = [2021, 2020, 2019, 2018]

if 2021 >= a :
    print(a)

TypeError: '>=' not supported between instances of 'int' and 'list'